## Create a list of identifiers for current laws and articles

In [1]:
# Load xlsx file 
import pandas as pd

import re
import hashlib

df = pd.read_excel('/Users/alexa/Projects/cdmx_kg/Mexico_City/Mexico city.xlsx')

df.head()


,city,type_document,type_document_2,juridiction_level,juridiction_level_name,document_name,document_start_date,document_finished_date,document_section_title,text,domains,row_id,doc_hash,document_name.1,document_section_title.1
0,Mexico City,law,constitution,city_municipality,Mexico City,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,2024,current,Artículo 1,De la Ciudad de México\n\n1. \tLa Ciudad de Mé...,governance,F823AF8C_ARTCULO1,F823AF8C,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,Artículo 1
1,Mexico City,law,constitution,city_municipality,Mexico City,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,2024,current,Artículo 2,"De la naturaleza intercultural, pluriétnica, p...","governance, culture_and_recreation, social_coh...",F823AF8C_ARTCULO2,F823AF8C,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,Artículo 2
2,Mexico City,law,constitution,city_municipality,Mexico City,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,2024,current,Artículo 3,De los principios rectores\n\n1. \tLa dignidad...,"governance, social_cohesion",F823AF8C_ARTCULO3,F823AF8C,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,Artículo 3
3,Mexico City,law,constitution,city_municipality,Mexico City,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,2024,current,Artículo 4,Principios de interpretación y aplicación de l...,governance,F823AF8C_ARTCULO4,F823AF8C,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,Artículo 4
4,Mexico City,law,constitution,city_municipality,Mexico City,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,2024,current,Artículo 5,Ciudad garantista\n\nA. Progresividad de los d...,governance,F823AF8C_ARTCULO5,F823AF8C,CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO,Artículo 5


In [3]:
# ID Generation Functions
def create_document_hash(document_name):
    """Create a short 8-character hash from document name"""
    if pd.isna(document_name):
        return "UNKNOWN"
    # Create MD5 hash and take first 8 characters
    hash_obj = hashlib.md5(str(document_name).encode('utf-8'))
    return hash_obj.hexdigest()[:8].upper()

def generate_scalable_row_id(row_index, document_name, section_title="", prefix="CDMX"):
    """Generate scalable IDs for multiple legal documents"""
    doc_hash = create_document_hash(document_name)
    section_clean = re.sub(r'[^\w\d]', '', str(section_title))[:6].upper() if section_title else "NOSEC"
    return f"{prefix}_{doc_hash}_{section_clean}_{row_index:05d}"

def generate_extraction_id(extraction_type, doc_hash, sequential_num):
    """Generate IDs for extracted data (articles, entities)"""
    return f"{extraction_type}_{doc_hash}_{sequential_num:05d}"


In [4]:
# 1. Add document hash column (calculate once per document)
df['doc_hash'] = df['document_name'].apply(create_document_hash)

# 2. Add cleaned section column
df['section_clean'] = df['document_section_title'].apply(
    lambda x: re.sub(r'[^a-zA-Z0-9]', '', str(x))[:13].upper() if pd.notna(x) else "NOSEC"
    #lambda x: re.sub(r'[^\w\d]', '', str(x))[:13].upper() if pd.notna(x) else "NOSEC"
)

# 3. Generate final row IDs using pre-calculated components
df['row_id'] = df.apply(
    lambda row: f"{row['doc_hash']}_{row['section_clean']}", axis=1
)

# Show document hash for reference
doc_hash = df['doc_hash'].iloc[0]

df = df[['row_id', 'doc_hash', 'document_name','document_section_title','text']]

df.to_csv('/Users/alexa/Projects/cdmx_kg/data/identifiers_0.csv', index=False, encoding='utf-8-sig')
print(df.head())

              row_id  doc_hash                                 document_name  \
0  F823AF8C_ARTCULO1  F823AF8C  CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO   
1  F823AF8C_ARTCULO2  F823AF8C  CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO   
2  F823AF8C_ARTCULO3  F823AF8C  CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO   
3  F823AF8C_ARTCULO4  F823AF8C  CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO   
4  F823AF8C_ARTCULO5  F823AF8C  CONSTITUCIÓN POLÍTICA DE LA CIUDAD DE MÉXICO   

  document_section_title                                               text  
0             Artículo 1  De la Ciudad de México\n\n1. \tLa Ciudad de Mé...  
1             Artículo 2  De la naturaleza intercultural, pluriétnica, p...  
2             Artículo 3  De los principios rectores\n\n1. \tLa dignidad...  
3             Artículo 4  Principios de interpretación y aplicación de l...  
4             Artículo 5  Ciudad garantista\n\nA. Progresividad de los d...  
